# <u> Memory-based Collaborative Filtering</u>

This notebook explores **memory-based collaborative filtering** as a first baseline for building a movie recommendation system. The approach relies solely on the raw user–item rating matrix, without the need for training or any machine learning framework.

The general idea is to build recommendations based on similarities, which can be computed in two main ways:

- **User–User Collaborative Filtering**: recommends movies liked by users who have similar preferences and rating behavior to us.
- **Item–Item Collaborative Filtering**: recommends new movies that are similar to the ones we have already seen and liked.

<br>

To define similarities between users or items, we commonly use two metrics:

- **Cosine Similarity**: a fast and popular measure that computes the angle between two vectors (single users or items). The formula is:

<br>

$$
\text{sim}_{\text{cosine}}(x, y) = \frac{x \cdot y}{\|x\| \|y\|}
$$

<br>

- **Pearson Correlation**: more computationally demanding, but it measures the linear relationship between co-rated items, correcting for each user’s individual rating bias. The formula is:

<br>

$$
\text{sim}_{\text{pearson}}(x, y) = \frac{\sum_{i}(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum(x_i - \bar{x})^2} \sqrt{\sum(y_i - \bar{y})^2}}
$$

<br>

Due to its ability to correct for individual rating biases, Pearson correlation is particularly effective in user–user collaborative filtering, where users may differ significantly in how they rate items. However, in item–item collaborative filtering, this adjustment is not appropriate, as centering on item means does not address user-level biases. Instead, cosine similarity and its adjusted form,are used to account for user rating behavior when comparing items.


## <u>0. Setting:</u>

### <u>0.1 Import libraries and dataframe</u>

In [ ]:
# Import necessary libraries
import pandas as pd, numpy as np, os, sys, seaborn as sns, matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.sparse import csr_matrix
from scipy.stats import pearsonr
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
import pyarrow as pa
import pyarrow.parquet as pq
from surprise import Dataset, Reader, KNNWithMeans, accuracy
import time

# Set the working directory
current_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(current_dir, ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import module for data processing
from modules.data_analysis import *


In [2]:
# Import cleand dataframe
file_path = '../data/processed/ratings_processed.parquet'
ratings_processed = pd.read_parquet(file_path, engine="pyarrow")
ratings_processed.head(3)

,userId,movieId,rating,timestamp,movie_bayes_avg,log_count_review,release_year,user_avg_rating,user_avg_bayes
0,1,2,3.5,2005-04-02 23:53:47,3.212004,10.009828,1995,3.742857,3.630537
1,1,29,3.5,2005-04-02 23:31:16,3.950567,9.050289,1995,3.742857,3.630537
2,1,32,3.5,2005-04-02 23:33:39,3.897763,10.713995,1995,3.742857,3.630537


In [3]:
print(f"Number of unique userId: {ratings_processed['userId'].nunique()}")
print(f"Number of unique movieId: {ratings_processed['movieId'].nunique()}")
print(f"Number of reviews: ~ {len(ratings_processed)/1000000:.1f} M")

Number of unique userId: 138383
Number of unique movieId: 12531
Number of reviews: ~ 19.9 M


### <u>0.2 Model evaluation</u>

To evaluate model performance and enable fair comparison with future approaches, we adopt a **train-validation-test split**: for each user, the last 4 ratings are held out for testing, while the 5th and 6th most recent ratings are used for validation. The remaining ratings are used for training the model. This setup mirrors real-world scenarios where unseen items are recommended based on past behavior, while the validation set is used for hyperparameter tuning to improve model generalization.

Model accuracy is assessed using the **Root Mean Squared Error (RMSE)** between predicted and actual ratings on the held-out items. Taking the square root of the Mean Squared Error ensures that the metric is expressed on the same scale as the original ratings, providing an interpretable measure of prediction error.

Holding out the 2–4 most recent ratings per user provides a good balance between training size and evaluation coverage across the entire dataset. Since we only include users with at least 20 ratings, this results in a minimum of approximately **70% train, 10% validation, and 20% test split per user**. Although the percentage of held-out ratings per user is small, the large size of the dataset ensures a substantial number of data points, enabling robust and reliable validation and evaluation across the user base.



In [6]:
# Sort in term of review date and user id and then pick 4 most recent review for each userId to build the test set 
sorted_df = ratings_processed.sort_values(by=['userId','timestamp'], ascending=[True,True])
test_df = sorted_df.groupby('userId').tail(4)

# Pick the 5th and 6th most recent reviews for each userId to build the validation set
val_df = sorted_df.groupby('userId').tail(6).groupby('userId').head(2)

# Build train df by removing the test_df and val_df rows from it
train_df = ratings_processed.drop(test_df.index).drop(val_df.index).reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

# Select only necessary columns
train_df = train_df[['userId', 'movieId', 'rating']]
test_df = test_df[['userId', 'movieId', 'rating']]
val_df = val_df[['userId', 'movieId', 'rating']]

In [7]:
# Assert correct length across users in the validation and test set
assert test_df.groupby('userId').size().eq(4).all(), "Some users in test_df have ≠ 4 ratings"
assert val_df.groupby('userId').size().eq(2).all(), "Some users in val_df have ≠ 2 ratings"

In [8]:
# Find length of train, validation, and test set for a given userId
user_id = 1
user_1_train_len = train_df[train_df['userId'] == user_id].shape[0]
user_1_val_len = val_df[val_df['userId'] == user_id].shape[0]
user_1_test_len = test_df[test_df['userId'] == user_id].shape[0]
print(f"UserId {user_id} - Train set length: {user_1_train_len}")
print(f"UserId {user_id} - Validation set length: {user_1_val_len}")
print(f"UserId {user_id} - Test set length: {user_1_test_len}")

# print sample of train, validation, and test set for userId 
print(f"\nTrain set sample for userId = {user_id}:")
print(train_df[train_df['userId'] == user_id].head(3))
print(f"\nValidation set sample for userId = {user_id}:")
print(val_df[val_df['userId'] == user_id].head(2))
print(f"\nTest set sample for userId = {user_id}:")
print(test_df[test_df['userId'] == user_id].head(4)) 


UserId 1 - Train set length: 169
UserId 1 - Validation set length: 2
UserId 1 - Test set length: 4

Train set sample for userId = 1:
   userId  movieId  rating
0       1        2     3.5
1       1       29     3.5
2       1       32     3.5

Validation set sample for userId = 1:
   userId  movieId  rating
0       1     1525     3.0
1       1     5999     3.5

Test set sample for userId = 1:
   userId  movieId  rating
0       1     7449     3.5
1       1     4133     3.0
2       1     3997     3.5
3       1     1750     3.5


## <u> 1. User-User Collaborative Filtering </u>

As previously explained, the main idea behind the **user–user collaborative filtering** approach is to recommend items to a target user by finding other users with similar tastes, then suggesting items they liked but the target user hasn't seen yet.

To compute similarities across users, we use Pearson **correlation** on the **user–item rating matrix**, which is structured as follows:
- Rows represent users (`userId`)
- Columns represent movies (`movieId`)
- Values are the ratings users assigned to each movie

This results in a very sparse matrix, as the dataset contains approximately 139,000 users and 12,000 movies, and most users rate only a small fraction of all available movies.

The prediction for how much a user $u$ will like an unseen item $i$ is computed using a **similarity-weighted average** of the ratings from the most similar users who have rated item $i$:

$$
\hat{r}_{u,i} = \bar{r}_u + \frac{\sum_{v \in N(u)} \text{sim}(u,v) \cdot (r_{v,i} - \bar{r}_v)}{\sum_{v \in N(u)} |\text{sim}(u,v)|}
$$

Where:
- $\hat{r}_{u,i}$ is the predicted rating for user $u$ on item $i$
- $\bar{r}_u$ is the average rating of user $u$
- $N(u)$ is the set of top-$K$ most similar users to $u$ who rated item $i$
- $\text{sim}(u,v)$ is the Pearson correlation between users $u$ and $v$
- $r_{v,i}$ is the rating that user $v$ gave to item $i$
- $\bar{r}_v$ is the average rating of user $v$

<br>

For the purpose of **hyperparameter tuning**, we evaluate different values of $K$, meaning that predictions are based on the top-$K$ most similar users who have rated each item. The optimal value of $K$ balances the trade-off between smoothing out noise from individual ratings and focusing on the most relevant users. Additionally, by setting a **minimum of 25 reviews per movie**, we ensure that each item has sufficient rating data to support reliable top-$K$ neighbor selection during prediction.

<br>

### <u> 1.1 Data Preparation for surprise</u>

In [ ]:
# Define rating scale (0.5-5)
reader = Reader(rating_scale=(0.5,5))

# Load training set into Surprise dataset
trainset = Dataset.load_from_df(train_df, reader).build_full_trainset()

# Prepare validation and test sets as lists of tuples
valset= [(row.userId, row.movieId, row.rating) for row in val_df.itertuples(index=False)]
testset = [(row.userId, row.movieId, row.rating) for row in test_df.itertuples(index=False)]

### <u> 1.2 Hyperparameter tuning </u>

In [ ]:
best_k = None
best_rmse = float('inf')

# Loop over different k values
for k in [5,10,20,30,40]:
    model = KNNWithMeans(k=k, sim_options = {"name":"pearson", "user_based":True})
    model.fit(trainset)
    pred_val = model.test(valset)
    rmse = accuracy.rmse(pred_val,verbose = False)

    if rmse < best_rmse:
        best_rmse = rmse
        best_k = k

# Print best hyperparameter along with the respective validation loss
print('Hyperparameter tuning for user-user collaborative filtering:')
print(f"Best k: {best_k} with validation RMSE:{best_rmse}")

### <u> 1.3 Model Evaluation </u>

In [ ]:
# Start time
start = time.perf_counter()

# Train final model with the best hyperparemeter
model = KNNWithMeans(k=best_k, sim_options = {"name":"pearson", "user_based":True})
model.fit(trainset)

# Evaluate final model on test set
pred_test = model.test(testset)
rmse_test_u_u = accuracy.rmse(pred_test, verbose = False)
mae_test_u_u = accuracy.mae(pred_test, verbose = False)

# Training and Evaluation time
elapsed_time_u_u = time.perf_counter() - start

# Summary model's performance
print(f"Model performance user-user collaborative filtering:\n RMSE: {rmse_test_u_u}\n MAE: {mae_test_u_u}\n Time: {elapsed_time_u_u} seconds.")

## <u> 2. Item–Item Collaborative Filtering </u>

As an alternative to user–user collaborative filtering, item–item collaborative filtering focuses on relationships between items rather than users. Instead of identifying users with similar preferences, recommendations are generated by comparing items previously rated by a user with the target item. This approach is generally more stable and computationally efficient, since items tend to receive more ratings than individual users, resulting in more reliable similarity estimates.

<br>

**<u>Similarity:</u>**

Item similarity is computed using **adjusted cosine similarity**, which corrects for user rating biases by centering each rating around the corresponding user’s average. This prevents misleading similarity scores caused by users who consistently rate higher or lower than others. For two items $i$ and $j$, adjusted cosine similarity is defined as:

<br>

$$
\text{sim}(i, j)
=
\frac{\sum_{u \in U_{ij}} (R_{ui} - \bar{R}_u) (R_{uj} - \bar{R}_u)}
{\sqrt{\sum_{u \in U_{ij}} (R_{ui} - \bar{R}_u)^2} \cdot 
 \sqrt{\sum_{u \in U_{ij}} (R_{uj} - \bar{R}_u)^2}}
$$

<br>

Where:  
- $U_{ij}$ denotes the set of users who have rated both items $i$ and $j$  
- $R_{ui}$ is the rating given by user $u$ to item $i$  
- $R_{uj}$ is the rating given by user $u$ to item $j$  
- $\bar{R}_u$ is the average rating given by user $u$, which removes user-specific bias  

<br>

**<u>Prediction:</u>**

A predicted rating $\hat{R}_{ui}$ for user $u$ on item $i$ is computed as a weighted average of the user’s deviations from the mean on items similar to $i$:

<br>

$$
\hat{R}_{ui}
=
\bar{R}_i +
\frac{\sum_{j \in N(i;u)} \text{sim}(i, j) \cdot (R_{uj} - \bar{R}_j)}
{\sum_{j \in N(i;u)} |\text{sim}(i, j)|}
$$

<br>

Where:  
- $\bar{R}_i$ is the average rating of item $i$ across all users  
- $N(i;u)$ denotes the set of the top $k$ most similar items to item $i$ that have been rated by user $u$  
- $\bar{R}_j$ is the average rating of item $j$  
- $\text{sim}(i,j)$ is the adjusted cosine similarity between items $i$ and $j$  
- $R_{uj}$ is the rating of user $u$ for item $j$  

<br>

To ensure stable and accurate predictions, the neighborhood size $k$ is **treated as a hyperparameter** and evaluated on the validation set. This allows the model to find an optimal trade-off between smoothing noise from individual ratings and focusing on the most relevant items.


### <u> 2.1 Data Preparation for surprise</u>

Given that the Surprise library does not natively implement adjusted cosine similarity, a correction for each user’s average rating is applied to the ratings before computing standard cosine similarity. This approach effectively reproduces the adjusted cosine similarity while maintaining the computational efficiency provided by the Surprise package. To avoid data leakage, the user means are computed on the training set only and then applied to the validation and test sets during prediction.


In [ ]:
# Compute mean_rating over train dataframe
user_means = train_df.groupby('userId')['rating'].mean()

# Centralise dataframes (train, validation, and test sets)
train_df_centered = train_df.copy()
train_df_centered['rating'] = train_df_centered.apply(lambda row: row['rating'] - user_means[row['userId']], axis=1)

val_df_centered = val_df.copy()
val_df_centered['rating'] = val_df_centered.apply(lambda row: row['rating'] - user_means[row['userId']], axis=1)

test_df_centered = test_df.copy()
test_df_centered['rating'] = test_df_centered.apply(lambda row: row['rating'] - user_means[row['userId']], axis=1)

# Define centered rating scale
min_rating = train_df_centered['rating'].min()
max_rating = train_df_centered['rating'].max()
reader = Reader(rating_scale=(min_rating,max_rating))

# Load training set into Surprise dataset
trainset = Dataset.load_from_df(train_df_centered, reader).build_full_trainset()

# Prepare validation and test sets as lists of tuples
valset = [(row.userId, row.movieId, row.rating) for row in val_df_centered.itertuples(index=False)]
testset = [(row.userId, row.movieId, row.rating) for row in test_df_centered.itertuples(index=False)]


### <u> 2.2 Hyperparameter tuning </u>

In [ ]:
best_k = None
best_rmse = float('inf')

# Loop over different k values
for k in [5,10,20,30,40]:
    model = KNNWithMeans(k=k, sim_options = {"name":"cosine", "user_based":False})
    model.fit(trainset)
    pred_val = model.test(valset)
    rmse = accuracy.rmse(pred_val,verbose = False)

    if rmse < best_rmse:
        best_rmse = rmse
        best_k = k

# Print best hyperparameter along with the respective validation loss
print('Hyperparameter tuning for item-item collaborative filtering:')
print(f"Best k: {best_k} with validation RMSE:{best_rmse}")

### <u> 2.3 Model Evaluation </u>

In [ ]:
# Start time
start = time.perf_counter()

# Train final model with the best hyperparemeter
model = KNNWithMeans(k=best_k, sim_options = {"name":"cosine", "user_based":True})
model.fit(trainset)

# Evaluate final model on test set
pred_test = model.test(testset)
rmse_test_i_i = accuracy.rmse(pred_test, verbose = False)
mae_test_i_i = accuracy.mae(pred_test, verbose = False)

# Training and Evaluation time
elapsed_time_i_i = time.perf_counter() - start

# Summary model's performance
print(f"Model performance item-item collaborative filtering: \n RMSE: {rmse_test_i_i}\n MAE: {mae_test_i_i}\n Time: {elapsed_time_i_i} seconds.")

## <u> 3. Summary of findings </u>

### <u> 3.1 Model performance </u>
